In [1]:
import tensorflow as tf
import cv2
import sys
import random
import time 
import numpy as np
from collections import deque
import pygame
from ple import PLE
from matplotlib import pyplot as plt

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [2]:
GAME  = "CATCHER"
ACTIONS = 3 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 500. # timesteps to observe before training
EXPLORE = 500. # frames over which to anneal epsilon
FINAL_EPSILON =  0.05 # final value of epsilon
INITIAL_EPSILON = 1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1

In [3]:
from ple.games.catcher import Catcher
#game = Catcher()
game = Catcher(width=256, height=256)
game.screen = pygame.display.set_mode(game.getScreenDims(), 0, 32)

In [4]:
# NUM_STEPS = 1000
# from ple import PLE
# p =  PLE(game,display_screen=True)
# actions = p.getActionSet()
# p.init()
# reward = p.act(p.NOOP)
# score  = 0
# for i in range(NUM_STEPS):
#     obs = p.getScreenRGB()
#     action_index =  np.random.randint(0, 3)
#     reward = p.act(actions[action_index ])
#     time.sleep(0.5)
#     #print(reward)
#     score += reward
#     if p.game_over():
#         p.reset_game()
#         print("time_step:",i,"score:",score)
#         score = 0

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.01, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = "SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

def createNetwork():
    # network weights
    W_conv1 = weight_variable([8, 8, 4, 32])
    b_conv1 = bias_variable([32])

    W_conv2 = weight_variable([4, 4, 32, 64])
    b_conv2 = bias_variable([64])

    W_conv3 = weight_variable([3, 3, 64, 64])
    b_conv3 = bias_variable([64])

    W_fc1 = weight_variable([1600, 512])
    b_fc1 = bias_variable([512])

    W_fc2 = weight_variable([512, ACTIONS])
    b_fc2 = bias_variable([ACTIONS])

    # input layer
    s = tf.placeholder("float", [None, 80, 80, 4])

    # hidden layers
    h_conv1 = tf.nn.relu(conv2d(s, W_conv1, 4) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, 2) + b_conv2)
    #h_pool2 = max_pool_2x2(h_conv2)

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 1) + b_conv3)
    #h_pool3 = max_pool_2x2(h_conv3)

    #h_pool3_flat = tf.reshape(h_pool3, [-1, 256])
    h_conv3_flat = tf.reshape(h_conv3, [-1, 1600])

    h_fc1 = tf.nn.relu(tf.matmul(h_conv3_flat, W_fc1) + b_fc1)

    # readout layer
    readout = tf.matmul(h_fc1, W_fc2) + b_fc2

    return s, readout, h_fc1

In [6]:
def trainNetwork(s, readout, h_fc1, sess):
    tick = time.time()
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices=1)
    cost = tf.reduce_mean(tf.square(y - readout_action))
    train_step = tf.train.AdamOptimizer(1e-6).minimize(cost)

    # open up a game state to communicate with emulator
    # game_state = game.GameState()
    p =  PLE(game,display_screen=True)
    actions = p.getActionSet()
    p.init()
    reward = p.act(p.NOOP)
    # store the previous observations in replay memory
    D = deque()

    # get the first state by doing nothing and preprocess the image to 80x80x4
    
    action_index =  np.random.randint(0, 3)
    reward = p.act(actions[action_index])
    if(reward != 0):
        if(reward < -1):
            r_0 = -1
        else:
            r_0 = reward
        terminal = 1
        if(np.max(readout_t) > qmax):
            qmax = np.max(readout_t) 
        print("time_step", t,"/SCORE", score,"/rt", r_t,"/ Q_MAX %e" % qmax)
        qmax = -10000
    else:
        terminal = 0
        r_0 = reward
            
    x_t = p.getScreenRGB()
    x_t = cv2.cvtColor(cv2.resize(x_t, (80, 80)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    x_t = x_t
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    sess.run(tf.global_variables_initializer())
    
    # saving and loading networks
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state("saved_networks")
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")

    # start training
    epsilon = INITIAL_EPSILON
    t = 0
    score = 0
    qmax = -100000
    while True:
        # choose an action epsilon greedily
        readout_t = readout.eval(feed_dict={s : [s_t]})[0]
        a_t = np.zeros([ACTIONS])
        action_index = 0
        if t % FRAME_PER_ACTION == 0:
            if random.random() <= epsilon:
                #print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[random.randrange(ACTIONS)] = 1
            else:
                action_index = np.argmax(readout_t)
                a_t[action_index] = 1
        else:
            a_t[0] = 1 # do nothing
        # scale down epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        reward = p.act(actions[action_index])
        if(reward != 0):
            if(reward < -1):
                r_t = -1
            else:
                r_t = reward
            terminal = 1
            if(np.max(readout_t) > qmax):
                qmax = np.max(readout_t) 
            print("time_step", t,"/SCORE", score,"/rt", r_t,"/ Q_MAX %e" % qmax)
            qmax = -10000
        else:
            terminal = 0
            r_t = reward
        # run the selected action and observe next state and reward
        x_t1_colored = p.getScreenRGB()
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored, (80, 80)), cv2.COLOR_BGR2GRAY)
        ret, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (80, 80, 1))
        #s_t1 = np.append(x_t1, s_t[:,:,1:], axis = 2)
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)

        # store the transition in D
        D.append((s_t, a_t, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        # only train if done observing
        if t > OBSERVE:
            # sample a minibatch to train on
            minibatch = random.sample(D, BATCH)

            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]

            y_batch = []
            readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            for i in range(0, len(minibatch)):
                terminal = minibatch[i][4]
                # if terminal, only equals reward
                if terminal:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

            # perform gradient step
            train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch}
            )

        # update the old values
        s_t = s_t1
        t += 1
        
        # save progress every 10000 iterations
        if t % 10000 == 0:
          saver.save(sess, 'saved_networks/' + GAME + '-dqn', global_step = t)

        score += reward
        if p.game_over():
            p.reset_game()
            #print("time_step:",t,"score:",score)
            score = 0

In [ ]:
def playGame():
    sess = tf.InteractiveSession()
    s, readout, h_fc1 = createNetwork()
    trainNetwork(s, readout, h_fc1, sess)

def main():
    playGame()

if __name__ == "__main__":
    main()

INFO:tensorflow:Restoring parameters from saved_networks/CATCHER-dqn-10000
Successfully loaded: saved_networks/CATCHER-dqn-10000
time_step 44 /SCORE 0 /rt -1.0 / Q_MAX 1.205970e-02
time_step 80 /SCORE -1.0 /rt -1.0 / Q_MAX 1.131311e-02
time_step 121 /SCORE -2.0 /rt -1 / Q_MAX 1.296398e-02
time_step 165 /SCORE 0.0 /rt 1.0 / Q_MAX 1.695568e-02
time_step 202 /SCORE 1.0 /rt -1.0 / Q_MAX 1.447841e-02
time_step 238 /SCORE 0.0 /rt 1.0 / Q_MAX 1.274469e-02
time_step 274 /SCORE 1.0 /rt 1.0 / Q_MAX 1.547485e-02
time_step 310 /SCORE 2.0 /rt -1.0 / Q_MAX 1.606860e-02
time_step 356 /SCORE 1.0 /rt -1 / Q_MAX 1.311548e-02
time_step 396 /SCORE 0.0 /rt 1.0 / Q_MAX 1.434636e-02
time_step 430 /SCORE 1.0 /rt -1.0 / Q_MAX 1.229268e-02
time_step 476 /SCORE 0.0 /rt -1.0 / Q_MAX 1.298404e-02
time_step 512 /SCORE -1.0 /rt -1 / Q_MAX 1.265128e-02
time_step 546 /SCORE 0.0 /rt 1.0 / Q_MAX 1.088810e-02
time_step 587 /SCORE 1.0 /rt 1.0 / Q_MAX 1.227705e-02
time_step 631 /SCORE 2.0 /rt -1.0 / Q_MAX 9.836440e-03
time

time_step 5722 /SCORE -2.0 /rt 1.0 / Q_MAX 1.854100e-02
time_step 5766 /SCORE -1.0 /rt -1 / Q_MAX -3.751274e-01
time_step 5803 /SCORE 0.0 /rt -1.0 / Q_MAX -4.370342e-01
time_step 5844 /SCORE -1.0 /rt 1.0 / Q_MAX -1.021482e-01
time_step 5885 /SCORE 0.0 /rt -1.0 / Q_MAX -3.887380e-01
time_step 5922 /SCORE -1.0 /rt -1 / Q_MAX -3.881010e-01
time_step 5962 /SCORE 0.0 /rt -1.0 / Q_MAX -3.887784e-01
time_step 6004 /SCORE -1.0 /rt -1.0 / Q_MAX -3.212925e-01
time_step 6050 /SCORE -2.0 /rt -1 / Q_MAX -3.964584e-01
time_step 6095 /SCORE 0.0 /rt -1.0 / Q_MAX -4.423128e-01
time_step 6129 /SCORE -1.0 /rt 1.0 / Q_MAX -8.593587e-02
time_step 6170 /SCORE 0.0 /rt -1.0 / Q_MAX -3.682036e-01
time_step 6214 /SCORE -1.0 /rt -1 / Q_MAX -4.167272e-01
time_step 6251 /SCORE 0.0 /rt -1.0 / Q_MAX -4.134550e-01
time_step 6297 /SCORE -1.0 /rt -1.0 / Q_MAX -3.493557e-01
time_step 6336 /SCORE -2.0 /rt -1 / Q_MAX -3.304953e-01
time_step 6374 /SCORE 0.0 /rt -1.0 / Q_MAX -4.159953e-01
time_step 6410 /SCORE -1.0 /rt -1.0

time_step 11333 /SCORE 5.0 /rt -1.0 / Q_MAX -5.565424e-01
time_step 11377 /SCORE 4.0 /rt -1.0 / Q_MAX -6.475872e-01
time_step 11423 /SCORE 3.0 /rt -1 / Q_MAX -5.823115e-01
time_step 11462 /SCORE 0.0 /rt 1.0 / Q_MAX -1.828986e-01
time_step 11508 /SCORE 1.0 /rt -1.0 / Q_MAX -3.500405e-01
time_step 11541 /SCORE 0.0 /rt 1.0 / Q_MAX 2.368492e-01
time_step 11574 /SCORE 1.0 /rt 1.0 / Q_MAX -7.160285e-03
time_step 11608 /SCORE 2.0 /rt 1.0 / Q_MAX 3.520560e-01
time_step 11649 /SCORE 3.0 /rt -1.0 / Q_MAX -3.558940e-01
time_step 11690 /SCORE 2.0 /rt 1.0 / Q_MAX 3.453629e-01
time_step 11732 /SCORE 3.0 /rt -1 / Q_MAX -7.619239e-02
time_step 11772 /SCORE 0.0 /rt 1.0 / Q_MAX 1.822616e-01
time_step 11812 /SCORE 1.0 /rt 1.0 / Q_MAX -1.185499e-01
time_step 11848 /SCORE 2.0 /rt 1.0 / Q_MAX 2.984441e-01
time_step 11887 /SCORE 3.0 /rt 1.0 / Q_MAX 3.285770e-01
time_step 11924 /SCORE 4.0 /rt -1.0 / Q_MAX -2.456607e-02
time_step 11957 /SCORE 3.0 /rt 1.0 / Q_MAX 1.507828e-01
time_step 11998 /SCORE 4.0 /rt 1.0 

time_step 16914 /SCORE 0.0 /rt -1.0 / Q_MAX -7.362776e-01
time_step 16950 /SCORE -1.0 /rt -1 / Q_MAX -4.988278e-01
time_step 16987 /SCORE 0.0 /rt -1.0 / Q_MAX -1.272634e-01
time_step 17020 /SCORE -1.0 /rt 1.0 / Q_MAX 7.304264e-01
time_step 17058 /SCORE 0.0 /rt 1.0 / Q_MAX 3.179898e-01
time_step 17101 /SCORE 1.0 /rt 1.0 / Q_MAX 1.595283e-01
time_step 17137 /SCORE 2.0 /rt 1.0 / Q_MAX 8.240638e-01
time_step 17181 /SCORE 3.0 /rt -1.0 / Q_MAX -7.206002e-01
time_step 17217 /SCORE 2.0 /rt 1.0 / Q_MAX 3.485305e-01
time_step 17256 /SCORE 3.0 /rt 1.0 / Q_MAX 3.625204e-01
time_step 17299 /SCORE 4.0 /rt 1.0 / Q_MAX 1.202574e-01
time_step 17342 /SCORE 5.0 /rt 1.0 / Q_MAX 2.705771e-01
time_step 17388 /SCORE 6.0 /rt -1 / Q_MAX -4.774453e-01
time_step 17427 /SCORE 0.0 /rt 1.0 / Q_MAX -2.660929e-01
time_step 17468 /SCORE 1.0 /rt -1.0 / Q_MAX -1.783365e-01
time_step 17501 /SCORE 0.0 /rt 1.0 / Q_MAX 7.099631e-01
time_step 17537 /SCORE 1.0 /rt 1.0 / Q_MAX 3.131170e-01
time_step 17583 /SCORE 2.0 /rt -1.0 /

time_step 22501 /SCORE 5.0 /rt -1.0 / Q_MAX -5.426431e-01
time_step 22539 /SCORE 4.0 /rt 1.0 / Q_MAX 4.440643e-01
time_step 22576 /SCORE 5.0 /rt -1 / Q_MAX -3.026038e-01
time_step 22618 /SCORE 0.0 /rt 1.0 / Q_MAX 5.227541e-01
time_step 22661 /SCORE 1.0 /rt 1.0 / Q_MAX 5.077467e-01
time_step 22705 /SCORE 2.0 /rt -1.0 / Q_MAX -2.768939e-01
time_step 22739 /SCORE 1.0 /rt -1.0 / Q_MAX -9.095550e-01
time_step 22775 /SCORE 0.0 /rt -1 / Q_MAX -1.622302e-01
time_step 22808 /SCORE 0.0 /rt 1.0 / Q_MAX 2.315407e-01
time_step 22842 /SCORE 1.0 /rt -1.0 / Q_MAX -8.340370e-01
time_step 22881 /SCORE 0.0 /rt 1.0 / Q_MAX 6.193936e-01
time_step 22916 /SCORE 1.0 /rt 1.0 / Q_MAX -4.031103e-02
time_step 22949 /SCORE 2.0 /rt 1.0 / Q_MAX 4.204002e-01
time_step 22986 /SCORE 3.0 /rt -1.0 / Q_MAX -9.790294e-01
time_step 23017 /SCORE 2.0 /rt 1.0 / Q_MAX 3.580158e-01
time_step 23053 /SCORE 3.0 /rt -1 / Q_MAX -4.401930e-01
time_step 23090 /SCORE 0.0 /rt -1.0 / Q_MAX -6.364129e-01
time_step 23126 /SCORE -1.0 /rt -1.

...

time_step 103926 /SCORE 132.0 /rt 1.0 / Q_MAX 1.089956e+00
time_step 103957 /SCORE 133.0 /rt 1.0 / Q_MAX 1.045357e+00
time_step 104000 /SCORE 134.0 /rt 1.0 / Q_MAX 8.593341e-01
time_step 104034 /SCORE 135.0 /rt 1.0 / Q_MAX 9.008486e-01
time_step 104075 /SCORE 136.0 /rt 1.0 / Q_MAX 1.058652e+00
time_step 104113 /SCORE 137.0 /rt 1.0 / Q_MAX 1.023691e+00
time_step 104156 /SCORE 138.0 /rt 1.0 / Q_MAX 1.108134e+00
time_step 104190 /SCORE 139.0 /rt 1.0 / Q_MAX 9.228681e-01
time_step 104231 /SCORE 140.0 /rt 1.0 / Q_MAX 9.756690e-01
time_step 104270 /SCORE 141.0 /rt 1.0 / Q_MAX 9.444425e-01
time_step 104306 /SCORE 142.0 /rt 1.0 / Q_MAX 9.387290e-01
time_step 104340 /SCORE 143.0 /rt 1.0 / Q_MAX 1.061512e+00
time_step 104373 /SCORE 144.0 /rt 1.0 / Q_MAX 1.087906e+00
time_step 104412 /SCORE 145.0 /rt 1.0 / Q_MAX 1.068550e+00
time_step 104448 /SCORE 146.0 /rt 1.0 / Q_MAX 9.758193e-01
time_step 104491 /SCORE 147.0 /rt 1.0 / Q_MAX 1.024674e+00
time_step 104534 /SCORE 148.0 /rt 1.0 / Q_MAX 1.008243e+

time_step 109034 /SCORE 271.0 /rt 1.0 / Q_MAX 9.904758e-01
time_step 109073 /SCORE 272.0 /rt 1.0 / Q_MAX 1.089473e+00
time_step 109109 /SCORE 273.0 /rt 1.0 / Q_MAX 9.792066e-01
time_step 109147 /SCORE 274.0 /rt 1.0 / Q_MAX 1.058324e+00
time_step 109180 /SCORE 275.0 /rt 1.0 / Q_MAX 9.558234e-01
time_step 109219 /SCORE 276.0 /rt 1.0 / Q_MAX 9.013315e-01
time_step 109257 /SCORE 277.0 /rt 1.0 / Q_MAX 1.066620e+00
time_step 109298 /SCORE 278.0 /rt 1.0 / Q_MAX 1.031993e+00
time_step 109337 /SCORE 279.0 /rt 1.0 / Q_MAX 1.068189e+00
time_step 109378 /SCORE 280.0 /rt 1.0 / Q_MAX 1.014155e+00
time_step 109411 /SCORE 281.0 /rt 1.0 / Q_MAX 9.948860e-01
time_step 109444 /SCORE 282.0 /rt 1.0 / Q_MAX 9.838386e-01
time_step 109480 /SCORE 283.0 /rt 1.0 / Q_MAX 9.854324e-01
time_step 109516 /SCORE 284.0 /rt 1.0 / Q_MAX 1.022506e+00
time_step 109547 /SCORE 285.0 /rt 1.0 / Q_MAX 9.959347e-01
time_step 109588 /SCORE 286.0 /rt 1.0 / Q_MAX 9.331444e-01
time_step 109626 /SCORE 287.0 /rt 1.0 / Q_MAX 9.255466e-

time_step 114158 /SCORE 410.0 /rt 1.0 / Q_MAX 9.971823e-01
time_step 114191 /SCORE 411.0 /rt 1.0 / Q_MAX 1.011989e+00
time_step 114222 /SCORE 412.0 /rt 1.0 / Q_MAX 9.308959e-01
time_step 114263 /SCORE 413.0 /rt 1.0 / Q_MAX 1.032441e+00
time_step 114296 /SCORE 414.0 /rt 1.0 / Q_MAX 9.888148e-01
time_step 114339 /SCORE 415.0 /rt 1.0 / Q_MAX 1.055274e+00
time_step 114375 /SCORE 416.0 /rt 1.0 / Q_MAX 1.044604e+00
time_step 114409 /SCORE 417.0 /rt 1.0 / Q_MAX 1.157416e+00
time_step 114440 /SCORE 418.0 /rt 1.0 / Q_MAX 8.510289e-01
time_step 114471 /SCORE 419.0 /rt 1.0 / Q_MAX 1.034087e+00
time_step 114507 /SCORE 420.0 /rt 1.0 / Q_MAX 1.068132e+00
time_step 114550 /SCORE 421.0 /rt 1.0 / Q_MAX 9.762177e-01
time_step 114583 /SCORE 422.0 /rt 1.0 / Q_MAX 1.013723e+00
time_step 114621 /SCORE 423.0 /rt 1.0 / Q_MAX 1.026307e+00
time_step 114654 /SCORE 424.0 /rt 1.0 / Q_MAX 1.029237e+00
time_step 114685 /SCORE 425.0 /rt 1.0 / Q_MAX 1.035942e+00
time_step 114719 /SCORE 426.0 /rt 1.0 / Q_MAX 1.052314e+

time_step 119220 /SCORE 549.0 /rt 1.0 / Q_MAX 9.067155e-01
time_step 119263 /SCORE 550.0 /rt 1.0 / Q_MAX 1.003725e+00
time_step 119296 /SCORE 551.0 /rt 1.0 / Q_MAX 1.039062e+00
time_step 119329 /SCORE 552.0 /rt 1.0 / Q_MAX 9.323914e-01
time_step 119367 /SCORE 553.0 /rt 1.0 / Q_MAX 1.061356e+00
time_step 119401 /SCORE 554.0 /rt 1.0 / Q_MAX 1.065463e+00
time_step 119434 /SCORE 555.0 /rt 1.0 / Q_MAX 9.990880e-01
time_step 119477 /SCORE 556.0 /rt 1.0 / Q_MAX 9.369839e-01
time_step 119516 /SCORE 557.0 /rt 1.0 / Q_MAX 9.650192e-01
time_step 119549 /SCORE 558.0 /rt 1.0 / Q_MAX 1.002059e+00
time_step 119590 /SCORE 559.0 /rt 1.0 / Q_MAX 1.057322e+00
time_step 119623 /SCORE 560.0 /rt 1.0 / Q_MAX 9.618757e-01
time_step 119664 /SCORE 561.0 /rt 1.0 / Q_MAX 1.001394e+00
time_step 119695 /SCORE 562.0 /rt 1.0 / Q_MAX 1.004176e+00
time_step 119726 /SCORE 563.0 /rt 1.0 / Q_MAX 9.060093e-01
time_step 119769 /SCORE 564.0 /rt 1.0 / Q_MAX 1.002486e+00
time_step 119808 /SCORE 565.0 /rt 1.0 / Q_MAX 1.121824e+

time_step 124405 /SCORE 686.0 /rt 1.0 / Q_MAX 1.040270e+00
time_step 124444 /SCORE 687.0 /rt 1.0 / Q_MAX 1.076596e+00
time_step 124485 /SCORE 688.0 /rt 1.0 / Q_MAX 9.860613e-01
time_step 124516 /SCORE 689.0 /rt 1.0 / Q_MAX 9.229696e-01
time_step 124547 /SCORE 690.0 /rt 1.0 / Q_MAX 9.872079e-01
time_step 124586 /SCORE 691.0 /rt 1.0 / Q_MAX 9.772013e-01
time_step 124619 /SCORE 692.0 /rt 1.0 / Q_MAX 1.033916e+00
time_step 124660 /SCORE 693.0 /rt 1.0 / Q_MAX 1.026623e+00
time_step 124698 /SCORE 694.0 /rt 1.0 / Q_MAX 9.298820e-01
time_step 124734 /SCORE 695.0 /rt 1.0 / Q_MAX 9.757365e-01
time_step 124773 /SCORE 696.0 /rt 1.0 / Q_MAX 9.140683e-01
time_step 124809 /SCORE 697.0 /rt 1.0 / Q_MAX 9.542022e-01
time_step 124850 /SCORE 698.0 /rt 1.0 / Q_MAX 9.580179e-01
time_step 124891 /SCORE 699.0 /rt 1.0 / Q_MAX 1.012628e+00
time_step 124924 /SCORE 700.0 /rt 1.0 / Q_MAX 1.031237e+00
time_step 124962 /SCORE 701.0 /rt 1.0 / Q_MAX 1.051405e+00
